In [ ]:
import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt

In [ ]:
# Constants and parameters from the original code
line_X = 0.2
omega_c = 50
Kp_pll = 0.2
Ki_pll = 5.0
Kp_i = 0.3
omega_0 = 1.0
omega_DQ = 1.0
omega_b = 2 * np.pi * 60
p0 = 0.7
q0 = 0.1
v0 = 1
mp = 100
mq = 0.05
Kp_vc = 1
Ki_vc = 2
Kf_vc = 1
Kp_ic = 1
Ki_ic = 2
Kf_ic = 0
Cf = 0.074
Lf = 0.08
Kv_i = Kp_i
V_inf = 1
theta_inf = 0

# Initial conditions
X0 = np.zeros(12)
X0[0] = p0  # P_hat
X0[1] = q0  # Q_hat
X0[4] = 0   # theta_t0 (initial angle)
X0[5] = 0   # delta
X0[8] = 1   # Vs_abs (initial voltage magnitude)



In [ ]:

# Define the ODE function for the inverter
def ODE_inverter(t, X, params):
    PLL_coeff, P_droop_Coeff, IVControl_coeff = params[:5], params[5:12], params[12:21]

    # Extract coefficients
    Kp_pll, Ki_pll, omega_0, omega_DQ, omega_b = PLL_coeff
    omega_c, Kp_i, p0, q0, v0, mp, mq = P_droop_Coeff
    Kp_vc, Ki_vc, Kf_vc, Kp_ic, Ki_ic, Kf_ic, Cf, Lf, Kv_i = IVControl_coeff
    
    # Unpack the state variables
    p_hat, q_hat, zeta, theta_pll, delta, phi_d, gamma_d, Vs_abs, Id_s, Iq_s, Vd_t, Vq_t = X
    
    # State-space transformations (DQ transformations)
    R = np.array([[np.cos(theta_pll), -np.sin(theta_pll)],
                  [np.sin(theta_pll),  np.cos(theta_pll)]])
    Vdq = np.array([Vd_t, Vq_t])
    VDQ = R @ Vdq
    
    VD_t = VDQ[0]
    VQ_t = VDQ[1]
    
    # Current calculations
    IQ_t = (V_inf - VD_t) / line_X
    ID_t = VQ_t / line_X
    Idq = np.linalg.solve(R, np.array([ID_t, IQ_t]))
    Id_t = Idq[0]
    Iq_t = Idq[1]
    
    # Power calculations
    p = Vd_t * Id_t + Vq_t * Iq_t
    q = Vq_t * Id_t - Vd_t * Iq_t
    
    # Differential equations for each state variable
    dX = np.zeros_like(X)
    dX[0] = omega_c * (p - p_hat)  # p_hat_dot
    dX[1] = omega_c * (q - q_hat)  # q_hat_dot
    theta_t = np.arctan2(VQ_t, VD_t)
    dX[2] = theta_t - theta_pll  # zeta_dot
    omega_pll = Kp_pll * (theta_t - theta_pll) + Ki_pll * zeta
    dX[3] = (omega_pll + omega_0 - omega_DQ) * omega_b  # theta_pll_dot

    p_star = p0 - mp * omega_pll
    dX[4] = Kp_i * (p_star - p_hat)  # delta_dot
    v_star = v0 - mq * (q_hat - q0)
    
    dX[5] = v_star - Vd_t  # phi_d_dot
    Id_s_hat = Kp_vc * (v_star - Vd_t) + Ki_vc * phi_d + Kf_vc * Id_t - (omega_pll + omega_0) * Cf * Vq_t
    dX[6] = Id_s_hat - Id_s  # gamma_d_dot
    
    Vd_s_hat = Kp_ic * (Id_s_hat - Id_s) + Ki_ic * gamma_d - Kf_ic * Vd_t - (omega_pll + omega_0) * Lf * Iq_s
    Vt_mag = np.sqrt(Vd_t**2 + Vq_t**2)
    dX[7] = Kv_i * (v_star - Vt_mag)  # Vs_abs_dot
    
    Vd_s = Vs_abs * np.cos(delta - theta_pll)
    Vq_s = Vs_abs * np.sin(delta - theta_pll)
    
    dX[8] = (omega_b * (Vd_s - Vd_t)) / Lf + (omega_pll + omega_0) * omega_b * Iq_s  # Id_s_dot
    dX[9] = (omega_b * (Vq_s - Vq_t)) / Lf - (omega_pll + omega_0) * omega_b * Id_s  # Iq_s_dot
    dX[10] = (omega_b * (Id_s - Id_t)) / Cf + (omega_pll + omega_0) * omega_b * Vq_t  # Vd_t_dot
    dX[11] = (omega_b * (Iq_s - Iq_t)) / Cf - (omega_pll + omega_0) * omega_b * Vd_t  # Vq_t_dot

    return dX

In [ ]:

# Parameters
PLL_coeff = [Kp_pll, Ki_pll, omega_0, omega_DQ, omega_b]
P_droop_Coeff = [omega_c, Kp_i, p0, q0, v0, mp, mq]
IVControl_coeff = [Kp_vc, Ki_vc, Kf_vc, Kp_ic, Ki_ic, Kf_ic, Cf, Lf, Kv_i]
params = PLL_coeff + P_droop_Coeff + IVControl_coeff + [V_inf]

# Time span for simulation
t_span = (0.0, 1.0)
t_eval = np.linspace(t_span[0], t_span[1], 1000)

# Solving the ODE system
sol = solve_ivp(ODE_inverter, t_span, X0, args=([params],), t_eval=t_eval, method='RK45')



In [ ]:
# Plotting the results
plt.figure(figsize=(10, 8))
plt.plot(sol.t, sol.y[0], label='P_hat (p_hat)')
plt.plot(sol.t, sol.y[1], label='Q_hat (q_hat)')
plt.plot(sol.t, sol.y[3], label='Theta_pll (theta_pll)')
plt.xlabel('Time (s)')
plt.ylabel('State Variables')
plt.title('GFM Inverter State Dynamics')
plt.legend()
plt.grid(True)
plt.show()